kernal：核函数类型。一般常用的有‘rbf’，‘linear’，‘poly’，等，发现使用‘rbf’参数时函数模型的拟合效果最好，其他拟合都是线性拟合
C：惩罚因子。C表征你有多么重视离群点，C越大越重视，越不想丢掉它们。C值大时对误差分类的惩罚增大，C值小时对误差分类的惩罚减小。当C越大，趋近无穷的时候，表示不允许分类误差的存在，margin越小，容易过拟合；当C趋于0时，表示我们不再关注分类是否正确，只要求margin越大，容易欠拟合。
gamma：是’rbf’，’poly’和’sigmoid’的核系数且gamma的值必须大于0。随着gamma的增大，存在对于测试集分类效果差而对训练分类效果好的情况，并且容易泛化误差出现过拟合

In [4]:
from sklearn.svm import SVR

svr = SVR(kernel='rbf', C=1e3, gamma=0.01)

In [7]:
import pandas as pd
df = pd.read_csv("./data2/handle_Turbine_Data .csv")

df_for_training = df[:-9685]
df_for_testing = df[-9685:]

In [9]:
'''
可以注意到数据范围非常大，并且它们没有在相同的范围内缩放，
因此为了避免预测错误，让我们先使用MinMaxScaler缩放数据。
(也可以使用StandardScaler)
'''
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
df_for_training_scaled = scaler.fit_transform(df_for_training)
df_for_testing_scaled = scaler.transform(df_for_testing)

In [10]:
def createXY(dataset, n_past):
    dataX = []
    dataY = []
    for i in range(n_past, len(dataset)):
        dataX.append(dataset[i - n_past:i, 0:dataset.shape[1]])
        dataY.append(dataset[i, 0])
    return np.array(dataX), np.array(dataY)

In [34]:
trainX, trainY = createXY(df_for_training_scaled, 60)
testX, testY = createXY(df_for_testing_scaled, 60)

In [67]:
trainX = np.array(df_for_training.iloc[:,1:])
trainY = np.array(df_for_training.iloc[:,-1:])
testX = np.array(df_for_testing.iloc[:,1:])
testY = np.array(df_for_testing.iloc[:,-1:])
svr_model = SVR(kernel='rbf',C=100,gamma=.001).fit(trainX,trainY)

E:\anaconda\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [68]:
print ('Testing R^2 =', round(svr_model.score(testX,testY),3))

Testing R^2 = 0.964


In [69]:
pred = svr_model.predict(testX)

In [70]:
pred

array([3.36767652, 4.32357049, 3.56030123, ..., 3.31918886, 3.32322719,
       3.49451079])

In [71]:
testY

array([[3.27017211],
       [4.30408479],
       [3.57908879],
       ...,
       [3.33183875],
       [3.28446822],
       [3.47520505]])

In [74]:
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
##########evaluate##############
# calculate MSE 均方误差 ---> E[(预测值-真实值)^2] (预测值减真实值求平方后求均值)
mse = mean_squared_error(pred, testY)
# calculate RMSE 均方根误差--->sqrt[MSE]    (对均方误差开方)
rmse = math.sqrt(mean_squared_error(pred, testY))
# calculate MAE 平均绝对误差----->E[|预测值-真实值|](预测值减真实值求绝对值后求均值）
mae = mean_absolute_error(pred, testY)
mape = mean_absolute_percentage_error(pred, testY)
print('均方误差: %.6f' % mse)
print('均方根误差: %.6f' % rmse)
print('平均绝对误差: %.6f' % mae)
print('平均绝对百分误差: %.6f' % mape)

均方误差: 0.135460
均方根误差: 0.368049
平均绝对误差: 0.119101
平均绝对百分误差: 0.021031
